In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.spatial import distance
from scipy.cluster import hierarchy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from scipy.cluster.hierarchy import fcluster
import process_data
from scipy.spatial import distance
from scipy.cluster import hierarchy
from sklearn.preprocessing import StandardScaler, MaxAbsScaler

ModuleNotFoundError: No module named 'umap'

In [3]:
data=process_data.process('all_data.csv')
X,y=process_data.split_x_y(data)

In [5]:
def patients_corr_analysis(data):
    X,y=process_data.split_x_y(data)
    CorrPatients=np.corrcoef(X)
    #cluster patients according to correlations
    row_linkage = hierarchy.linkage(
    distance.pdist(CorrPatients), method='complete')
    col_linkage = hierarchy.linkage(
    distance.pdist(CorrPatients.T), method='complete')
    cl=sns.clustermap(CorrPatients, row_linkage=row_linkage,col_linkage=col_linkage,figsize=(20, 20), cmap="YlGnBu")
    return row_linkage,pd.DataFrame(data=X,index=data.index,columns=data.columns[:-1])

In [ ]:
linkage_max,X=patients_corr_analysis(data,'Max')

In [ ]:
linkage_std,X=patients_corr_analysis(data,'Standard')

In [6]:
#What is inside the clusters ? 

def get_clusters(X,linkage,nb_clusters):
    fl = fcluster(linkage,nb_clusters,criterion='maxclust')
    clusters=[]
    for i in range(nb_clusters):
        df=X.iloc[np.where(fl==i+1)]
        clusters.append(df)
    return clusters
def get_gene_stats(cluster):
    gene_expressions=pd.DataFrame(columns=['min','max','moyenne','écart_type',],index=cluster.columns)
    gene_expressions['min']=cluster[cluster.columns].min()
    gene_expressions['max']=cluster[cluster.columns].max()
    gene_expressions['moyenne']=cluster[cluster.columns].mean()
    gene_expressions['écart_type']=cluster[cluster.columns].std()
    return gene_expressions
def get_MRD_stats(cluster):
    return pd.concat([cluster,y],axis=1).groupby('MRD Response').count().iloc[:,1][0]/len(cluster)*100

def patients_clusters_analysis(X,linkage,nb_clusters):
    clusters=get_clusters(X,linkage,nb_clusters)
    for i in range (len(clusters)):
        print('Le pourcentage de patients qui sont détectés MRD- est : ',get_MRD_stats(clusters[i]),',le nombre de patients dans le cluster est : ',len(clusters[i]))
    pass

In [30]:
patients_clusters_analysis(X,linkage_max,3)

Le pourcentage de patients qui sont détectés MRD- est :  72.72727272727273 ,le nombre de patients dans le cluster est :  11
Le pourcentage de patients qui sont détectés MRD- est :  50.38167938931297 ,le nombre de patients dans le cluster est :  262
Le pourcentage de patients qui sont détectés MRD- est :  53.21100917431193 ,le nombre de patients dans le cluster est :  218


In [32]:
patients_clusters_analysis(X,linkage_std,2)

Le pourcentage de patients qui sont détectés MRD- est :  57.692307692307686 ,le nombre de patients dans le cluster est :  78
Le pourcentage de patients qui sont détectés MRD- est :  51.08958837772397 ,le nombre de patients dans le cluster est :  413


### Only on selected features

In [22]:
extracted_features=list(pd.read_csv('features_svc.csv').values[:,1])

In [ ]:
linkage,X=patients_corr_analysis(pd.concat([data[extracted_features],y],axis=1),'None')

In [28]:
patients_clusters_analysis(X=X,linkage=linkage,nb_clusters=6)

Le pourcentage de patients qui sont détectés MRD- est :  49.78165938864629 ,le nombre de patients dans le cluster est :  458
Le pourcentage de patients qui sont détectés MRD- est :  100.0 ,le nombre de patients dans le cluster est :  9
Le pourcentage de patients qui sont détectés MRD- est :  75.0 ,le nombre de patients dans le cluster est :  16
Le pourcentage de patients qui sont détectés MRD- est :  100.0 ,le nombre de patients dans le cluster est :  2
Le pourcentage de patients qui sont détectés MRD- est :  66.66666666666666 ,le nombre de patients dans le cluster est :  3
Le pourcentage de patients qui sont détectés MRD- est :  100.0 ,le nombre de patients dans le cluster est :  3
